# Source: Wikipedia - All Americans

In [1]:
#hide
import json
import core_constants as cc
import functions as fx
import pandas as pd
import sqlite3 as sql
import recordlinkage

# Set Notebook Settings

In [2]:
years = cc.get_defYears()
headers= cc.get_header()
dataset = 'AllAmerican'

# Get, Process the All American data from Wikipedia
#### Source: https://www.pro-football-reference.com/years/2017/draft.htm
> This page contains metadata of each draft pick, both in terms of draft position but also current pro stats.

In [ ]:
with open("..//scrapedData//aaSelections.json", "w", encoding="utf-8") as write_file:
                write_file.write(json.dumps(fx.handle_allAmerican(years, headers)))

# Clear DB
> Useful for a clean start.  This removes all of the records for this dataset from the following structures: SourcedPlayers, RecordLinks.  All of the Views auto-cleanse themselves.

In [ ]:
fx.clearDB(dataset)

# Save to DB
> This saves it to the SourcedPlayers Table

In [ ]:
fx.toDB_AllAmerican()

# Strict Matching
> This saves it to RecordLinking where ID == ID

In [ ]:
fx.literalLinking(dataset)

# Fuzzy Matching w/ Threshold

> This is automatically pushing fuzzy matches above a certain threshold into the DB without the need for review

In [3]:
fuzzyDF = fx.doFuzzyMatching(dataset, 'Sports247')

# Create the Annotation File

> This changes the dataframe into a MultiIndex data frame that the annotation function requires

In [5]:
conn = sql.connect(cc.databaseName) 
          
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM SourcedPlayers
                               WHERE KeyDataSet = 1
                               ''', conn)

df_247 = pd.DataFrame(sql_query, columns = ['IDYR', 'College', 'Year', 'Position'])
df_247.set_index('IDYR', append=False, inplace=True)
sql_query = pd.read_sql_query ('''
                               SELECT
                               *
                               FROM UnlinkedAllAmerican
                               ''', conn)

df_AllAmerican = pd.DataFrame(sql_query, columns = ['ID', 'College'])
df_AllAmerican.set_index('ID', append=False, inplace=True)

fuzzyMI = pd.MultiIndex.from_frame(fuzzyDF)
recordlinkage.write_annotation_file(
    "../Annotations/Annotations/annotation_allamerican.json",
    fuzzyMI[0:300],
    df_AllAmerican,
    df_247,
    dataset_a_name="AllAmerican",
    dataset_b_name="Master"
)

# Read in the Annotation File
> Take the resulting Annotation file after handling the processing and insert it into the right table

In [6]:
annotation = recordlinkage.read_annotation_file("..//Annotations//Results//result_ncaa.json")
try:
    annotation_dict = (annotation.links).to_flat_index()
except Exception as e:
    print(e)

## Insert Annotations to RecordLinks

In [7]:
for record in annotation_dict:
    #MAKE SURE YOU UPDATE THE THIRD VALUE TO THE CORRECT KEYDATASET!!
    Values = [record[0], record[1], 6, 1, 1]
    query = '''INSERT INTO RecordLinks(MasterID, TargetID, KeyDataSet, KeyLinkType, LinkConfidence)
        VALUES (?,?,?,?,?)'''
    
    conn = sql.connect(cc.databaseName)
    c = conn.cursor()
    
    c.execute(query, Values)
    conn.commit()
    
conn.close()